<a href="https://colab.research.google.com/github/esgryaznova/compling_nlp_hse_course/blob/master/homework_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 9. Предобученные трансформеры 

### Задание 1 (8 баллов).
Дообучите 3 предобученных модели на тех же данных, что и в семинаре. Сравните качество и выберите лучшую. 
Модели должны отличаться друг от друга не только параметрами ru/en/multilingual, base/large, cased/uncased. Все три выбранные модели должны работать лучше, чем бейзлайн без дообучения (т.е. если модель не работает, попробуйте другую).




In [ ]:
!pip install transformers sklearn pandas matplotlib

In [2]:
# стандартные библиотеки
import os, re
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
%matplotlib inline


# pytortch и huggingface 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from transformers import TFAutoModel
from transformers import AutoTokenizer
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [5]:
data = pd.read_csv('lenta_sample.csv')
data.dropna(subset=['topic', 'text'], inplace=True)

Distilbert-base-ru-cased

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('Geotrend/distilbert-base-ru-cased')
model_distilbert = TFAutoModel.from_pretrained('Geotrend/distilbert-base-ru-cased', trainable=True, from_pt=True)

In [8]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [9]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    X.append(ids[:512])

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

In [10]:
train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)

In [11]:
X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [31]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_distilbert({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_distilbert_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_distilbert_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,
                                    ), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [ ]:
model_distilbert_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2,
          epochs=20)

In [33]:
pred = model_distilbert_clf.predict(X_valid, batch_size=5).argmax(1)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_valid.argmax(1), pred, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00         1
Силовые структуры       0.67      1.00      0.80         2
        Экономика       0.50      1.00      0.67         2
           Бизнес       0.33      0.50      0.40         2
             Крым       0.00      0.00      0.00         0
          Легпром       0.00      0.00      0.00         1
              Мир       0.00      0.00      0.00         1
              Дом       0.00      0.00      0.00         2
         Из жизни       0.00      0.00      0.00         3
            Спорт       1.00      1.00      1.00         2
   Интернет и СМИ       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
  Наука и техника       0.50      0.33      0.40         3
      Бывший СССР       0.50      0.67      0.57         3
           Россия       0.00      0.00      0.00         2
       Библиотека       0.00      0.00      0.00       

Bert Ru Cased

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")
model_bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased", trainable=True, from_pt=True)

In [18]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    X.append(ids[:512])

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)

In [19]:
train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)
X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [20]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_bert({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_bert_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_bert_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [ ]:
model_bert_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2,
          epochs=20)

In [22]:
pred5 = model_bert_clf.predict(X_valid, batch_size=5).argmax(1)

In [23]:
print(classification_report(y_valid.argmax(1), pred5, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

   Интернет и СМИ       1.00      0.50      0.67         2
            Спорт       1.00      1.00      1.00         2
           Бизнес       0.40      1.00      0.57         2
              Мир       0.00      0.00      0.00         1
           Россия       1.00      0.50      0.67         2
         Ценности       1.00      0.50      0.67         2
Силовые структуры       0.50      0.50      0.50         2
         Культура       1.00      1.00      1.00         3
  Наука и техника       1.00      0.67      0.80         3
         Из жизни       0.33      0.33      0.33         3
      Бывший СССР       0.67      0.67      0.67         3
   69-я параллель       1.00      1.00      1.00         1
          Легпром       0.00      0.00      0.00         1
              Дом       1.00      1.00      1.00         2
        Экономика       1.00      0.50      0.67         2
             Крым       0.00      0.00      0.00       

Rubert-tiny-sentiment-balanced

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced")
model_rubert = TFAutoModel.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced", trainable = True, from_pt=True)

In [7]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [8]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    X.append(ids[:512])

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

In [9]:
train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic, random_state=0)
X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [10]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_rubert({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_rubert_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_rubert_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [ ]:
model_rubert_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2,
          epochs=20)

In [14]:
pred4 = model_rubert_clf.predict(X_valid, batch_size=5).argmax(1)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_valid.argmax(1), pred4, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

   Интернет и СМИ       0.00      0.00      0.00         2
            Спорт       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00         2
              Мир       0.00      0.00      0.00         1
           Россия       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
Силовые структуры       0.00      0.00      0.00         2
         Культура       0.25      0.33      0.29         3
  Наука и техника       0.00      0.00      0.00         3
         Из жизни       0.33      0.67      0.44         3
      Бывший СССР       0.40      0.67      0.50         3
   69-я параллель       0.00      0.00      0.00         1
          Легпром       0.00      0.00      0.00         1
              Дом       0.00      0.00      0.00         2
        Экономика       0.00      0.00      0.00         2
             Крым       0.00      0.00      0.00       

LaBSE

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("setu4993/smaller-LaBSE")
model_LaBSE = TFAutoModel.from_pretrained('setu4993/smaller-LaBSE', trainable=True, from_pt=True)

In [53]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [54]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)
    X.append(ids[:512])

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

In [57]:
input_word_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32,
                                     name="input_ids")

output = model_LaBSE({"input_ids":input_word_ids})
drop = tf.keras.layers.Dropout(0.3)(output[0][:, 0]) 
dense = tf.keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_LaBSE_clf = tf.keras.Model(inputs=input_word_ids, outputs=dense)


model_LaBSE_clf.compile(tf.optimizers.Adam(learning_rate=2e-6,), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [ ]:
model_LaBSE_clf.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2,
          epochs=20)

In [59]:
pred3 = model_LaBSE_clf.predict(X_valid, batch_size=5).argmax(1)

In [60]:
print(classification_report(y_valid.argmax(1), pred3, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00         1
Силовые структуры       0.00      0.00      0.00         2
        Экономика       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00         2
             Крым       0.00      0.00      0.00         0
          Легпром       0.00      0.00      0.00         1
              Мир       0.00      0.00      0.00         1
              Дом       0.00      0.00      0.00         2
         Из жизни       0.20      0.33      0.25         3
            Спорт       0.00      0.00      0.00         2
   Интернет и СМИ       0.00      0.00      0.00         2
         Ценности       0.00      0.00      0.00         2
  Наука и техника       0.00      0.00      0.00         3
      Бывший СССР       0.00      0.00      0.00         3
           Россия       0.00      0.00      0.00         2
       Библиотека       0.00      0.00      0.00       

Я перепробовала еще другие модели, но результат был такой же плохой, возможно они все были слишком большие и на маленьком количестве эпох не обучаются достаточно. Попробую снова позже(

### Задание  2 (2 балла)

Ответьте на следующие вопросы (напишите хотя бы по несколько предложений):

1) Чем ROBERTA отличается от BERT? В какой статье описана ROBERTA?

2) Чем модель T5 отличается от ROBERTA/BERT? 

1) Ссылка на статью - https://arxiv.org/pdf/1907.11692.pdf

Роберта обучалась дольше и на бОльшей выборке, без цели next sentence prediction, на последовательностях дольше, чем на которых обучалась BERT. Плюс паттерн маскирования токенов динамично меняется во время обучения.

2) BERT и ROBERTA - модели типа encoder-only. T5 - модель encoder-decoder. T5 умеет генерировать текст, а BERT и ROBERTA предсказывают класс
